In [11]:
import hashlib
import pandas as pd
from typing import List

In [12]:
df = pd.read_csv('/Users/alexey_macos/Downloads/user_channels_duplicate_rows.csv')
head_df = df.head()
print(head_df)

   channel_id     user_id     channel_name                 channel_link  \
0           1  1129111522      @south_work      https://t.me/south_work   
1         149  1129111522   @banki_broshka   https://t.me/banki_broshka   
2         150  1129111522      @meduzalive      https://t.me/meduzalive   
3         163  1006278099  @if_market_news  https://t.me/if_market_news   
4         166  6646383117      @meduzalive      https://t.me/meduzalive   

           addition_timestamp  is_active  \
0  2025-03-24 23:00:36.072071       True   
1  2025-03-03 17:01:39.599933      False   
2  2025-03-17 00:00:35.257657      False   
3  2025-03-22 09:57:00.340668       True   
4  2025-02-14 22:52:58.246097       True   

                              channel_topic  hashed_channel  
0  ["Образование","стажировки","стипендии"]             NaN  
1           ["Банки","финансы","экономика"]             NaN  
2           ["Политика","военные действия"]             NaN  
3        ["Новости","экономика","пол

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191 entries, 0 to 190
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   channel_id          191 non-null    int64  
 1   user_id             191 non-null    int64  
 2   channel_name        191 non-null    object 
 3   channel_link        191 non-null    object 
 4   addition_timestamp  191 non-null    object 
 5   is_active           191 non-null    bool   
 6   channel_topic       191 non-null    object 
 7   hashed_channel      0 non-null      float64
dtypes: bool(1), float64(1), int64(2), object(4)
memory usage: 10.8+ KB


In [10]:
def generate_channel_hash(channel_name: str) -> int:
    """Generate a SHA-256 hash in int8 format for Postgres DB."""
    hash_bytes = hashlib.sha256(channel_name.encode("utf-8")).digest()[:8]
    hash_int = int.from_bytes(hash_bytes, byteorder='big', signed=False)
    # signed изменён с True на False, что гарантирует возврат только положительных значений
    return hash_int % (2**63) # Ограничение до 63 бит


# channel_name = '@KarpovCourses'
# channel_topic = ["Технологии и IT","Apple"]
# hashed_channel = generate_channel_hash(channel_name)
# hashed_channel = generate_channel_hash(channel_name, channel_topic)
# print(f'Hashed channel: {hashed_channel}', f'\ntype of Hashed channel: {type(hashed_channel)}')

In [7]:
# Generate hashes for each unique channel name
channel_name_to_hash = {}
for channel_name in df['channel_name'].unique():
    channel_name_to_hash[channel_name] = generate_channel_hash(channel_name)

# update the 'hashed_channel_name' column
df['hashed_channel'] = df['channel_name'].map(channel_name_to_hash)

# replace the 'channel_id' column with the 'hashed_channel_name' column
df['channel_id'] = df['hashed_channel']

df.to_csv('/Users/alexey_macos/Downloads/user_channels_dublicate_rows_hashed_channel.csv', index=False)